In [4]:
import pysam
import time
import pickle
import os
from collections import defaultdict

In [5]:
#==========================================================================================================
fa_folder = "/oak/stanford/groups/arend/Xin/AssemblyProj/reference_align_2/Software_Xin/Aquila/source"     #The path of the folder which contains your fasta files
fa_name = "ref.fa"               #The fasta file you want to process
out_dir = "/oak/stanford/groups/arend/Xin/LiuYC/test3"            #The output folder direction (if not exist, the script will create one)
start = 21                           #The start chromosome
end = 21                            #The end chromosome.
kmer_len = 100                      #Length of the kmers generated
mapq_thres = 20                     #MAPQ filter threshold (20 recommended)
bowtie_thread = 20                  #The number of parallel search threads bowtie2 would use 
#==========================================================================================================

Configure variables above to meet your requirements, and run the following blocks one by one.

NOTICE: You need to install Bowtie2 before running this script.

In [6]:
def QualStr (kmer_len):
    #Generate a fake quality string, default k = 100
    qual = ''
    for i in range(kmer_len):
        qual = qual+'~'
    return qual

The QualStr function generates quality string for later use in generating .fq files. 

In [7]:
def GetGenome (fa_folder,fa_name):
    genome ={}# key = chrnum ; item = chrseq
    chro = []
    with open(fa_folder+fa_name) as f:
        for line in f:
            if line.startswith('>'):
                if chro:
                    genome[chrnum] = ''.join(chro)
                    chro = []
                    fw.close()
                chrnum = line[1:].split(' ')[0].strip('\n')
                fw = open(fa_folder+chrnum+'.fa','w')
                fw.write(line)
            else:
                chro.append(line.strip('\n'))
                fw.write(line)
        genome[chrnum] = ''.join(chro)
        chro = []
    return genome # key : chrnum (chr21) value : chrseq

GetGenome function splits the input fasta file by chromosome name, in other words, it generates some new fasta files and each of them only contains ONE chromosome. Meanwhile, GetGenome saves sequence information into the genome dictionary (key:chromosome name;value:chromosome sequence).

Although this function assumes that the input fasta file contains multiple chromosomes, it is pretty okey if it only has one.

In [8]:
def GenerateFqFile (chrseq,chrnum,qual,kmer_len):
    with open(chrnum+'/'+chrnum+'.fq','w') as fw:
        for n in range(len(chrseq)+1-kmer_len):
            seq = chrseq[n:n+kmer_len]
            if 'N' not in seq:
                fw.write('@%s\n%s\n+\n%s\n'%(n+1,seq,qual))

The fastq file content would be like:

    @123456 
    ATCGGTAC...
    +
    ~~~~~~~~...
    
 (123456 is the kmer position)
 
 and saved as chrnum.fq eg:chr21.fq

In [9]:
def Bowtie2Map (chrnum,fa_folder,bowtie_thread):
    thread = str(bowtie_thread)
    os.chdir(chrnum+'/')
    os.mkdir('./ref'+chrnum)
    os.chdir('./ref'+chrnum)
    #index folder is refchrnum eg:refchr21
        
    index_build = os.popen('bowtie2-build --threads '+thread +' '+fa_folder+chrnum+'.fa'+' ref'+chrnum,'r')
    print(index_build.read())
        
    map_result = os.popen('bowtie2 -p '+thread+' -x '+'ref'+chrnum+' -U '+'../'+chrnum+'.fq '+'-S ../'+chrnum+'.sam','r')
    print(map_result.read())
    #output is chrnum.sam eg:chr21.sam
        
    os.chdir('..')
    out1 = os.popen('samtools view -bS '+chrnum+'.sam'+' > '+chrnum+'.bam')
    print(out1.read())
    out2 = os.popen('samtools sort '+chrnum+'.bam -o '+chrnum+'.sorted.bam')
    print(out2.read())
    out3 = os.popen('samtools view -H '+chrnum+'.sorted.bam > header.sam')
    print(out3.read())
    out4 = os.popen('samtools view -F 4 '+chrnum+'.sorted.bam | grep -v "XS:" | cat header.sam - | samtools view -b - > unique'+chrnum+'.bam')
    print(out4.read())
    out5 = os.popen('rm header.sam')
    print(out5.read())
    out6 = os.popen('samtools index unique'+chrnum+'.bam')
    print(out6.read())
    os.chdir('..')

Bowtie2Map uses Bowtie2 to get mapping result, and saves unique mapped reads to uniquechrnum.bam (eg:uniquechr1.bam).

CAUTION: Make sure you have already installed samtools and Bowtie2 package!

In [10]:
def Filter(chrnum,mapq_thres,kmer_len):
    filtered = []
    bamfile = pysam.AlignmentFile(chrnum+'/'+'unique'+chrnum+'.bam',"rb")
    for read in bamfile:
        if int(read.mapq) >= mapq_thres and not (read.is_reverse):
            filtered.append([int(read.pos),int(read.pos)+kmer_len-1])
    return filtered

Filter function filters out reads which are reverse or MAPQ lower than threshold

In [11]:
def Merge(chrnum,filtered):
    start = 0
    with open(chrnum+'/'+'merged.bed',"w") as fm:
        for line in filtered:
            if start == 0:
                start,end = line[0],line[1]
            elif line[0] > end+1:
                fm.write("%s\t%s\t%s\n"%(chrnum,start,end))
                start,end = line[0],line[1]
            else:
                end = line[1] 
        fm.write("%s\t%s\t%s\n"%(chrnum,start,end))

Merge function merges those overlapping unique mapped kmers into bigger blocks

In [12]:
def Get_uniqness(chrnum):
    uniq_map = defaultdict(int)
    with open(chrnum+'/'+"merged.bed","r") as f:
        with open(chrnum+'/'+"500merged.bed","w") as fw:
            for line in f:
                data = line.rsplit()
                _start = int(data[1])
                _end = int(data[2])
                block_len = _end - _start
                if block_len >= 500:
                    use_start = _start + 10
                    use_end = _end - 10
                    fw.write('%s\t%s\t%s\n'%(chrnum,use_start, use_end))
                    for step in range(use_start, use_end+1):
                        uniq_map[step] = 1
    pickle.dump(uniq_map,open("Uniqness_map/"+"uniq_map_"+chrnum+".p","wb"))

Get_uniqness function uses the merge results to get unique mapped keys and saves them into a pickle file, which is the final output.

In [ ]:
def run(fa_folder,out_dir,chrseq,chrnum,kmer_len,qual,mapq_thres,bowtie_thread):
    print("Starting to process "+chrnum)
    t = time.time()
    os.mkdir(chrnum)
    
    GenerateFqFile (chrseq,chrnum,qual,kmer_len)
    print(chrnum,":Generate .fq DONE")
    Bowtie2Map (chrnum,fa_folder,bowtie_thread)
    print(chrnum,":Bowtie2 mapping DONE")
    filtered = Filter(chrnum,mapq_thres,kmer_len)
    print(chrnum,":MAPQ filter DONE")
    Merge(chrnum,filtered)
    print(chrnum,":Merge DONE")
    Get_uniqness(chrnum)
    print(chrnum,":Get uniqness DONE")
    #-----------------------------------------------------------------------------------------------
    out7 = os.popen('rm -R '+chrnum+'/')#These two lines delete the intermediate results. 
    print(out7.read())                   #If you want to keep those results, comment out these two.
    #-----------------------------------------------------------------------------------------------
    print(chrnum,"DONE! Time used:", time.time()-t)

In [ ]:
fa_folder = fa_folder+"/"
out_dir = out_dir+"/"
chr_start = start - 1
chr_end = end
qual = QualStr(kmer_len)
Genome = GetGenome(fa_folder,fa_name)
chr_list = list(Genome.keys())
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
os.chdir(out_dir)
    
if not os.path.exists('Uniqness_map'):
    os.mkdir('Uniqness_map')

for chrnum in chr_list[chr_start:chr_end]:
    chrseq = Genome[chrnum]
    run(fa_folder,out_dir,chrseq,chrnum,kmer_len,qual,mapq_thres,bowtie_thread)

for chrnum in chr_list:
    os.popen('rm '+fa_folder+chrnum+'.fa')

print("ALL DONE")

Starting to process chr21
chr21 :Generate .fq DONE
